#### This notebook is used for extracting and formatting data for publishing.

### View and save the unmatched images

In [1]:
with open(os.path.join(rendered_dir, 'unmatched_filenames.txt'), 'r') as f:
    files = []
    for fname in f:
        files.append(os.path.basename(fname.strip()))
    
df_bad = df.loc[files]
targets=[]; preds=[]; index=[]
for row in df_bad[['target_seq', 'pred_seq']].sample(55).itertuples(index=True):
    index.append(row[0])
    targets.append('$%s$'%row[1])
    preds.append('$%s$'%row[2])
df_ = pd.DataFrame(data={'gold': targets, 'pred': preds})
df_

NameError: name 'os' is not defined

### In favor of compatibility with publishing tools, remove entries that are not rendered by MathJax 
Though they rendered fine with render_latex.py above, MathJax has difficulty rendering them. Therefore we will remove them for visualization purposes just in case it causes a problem with publishing tools downstream.

In [2]:
df_2_ = df_.drop([51, 41, 18, 21, 35, 38, 26]).reindex()[:50]

NameError: name 'df_' is not defined

In [ ]:
pd.options.display.latex.repr = True
with open('/tmp/df_2_frag.tex', 'w') as f:
    f.write(df_2_.to_latex(escape=False))

# End

In [ ]:
num2display = 10
# with open(os.path.join(rendered_dir, 'unmatched_filenames.txt'), 'r') as f:
#     files = np.random.choice(f.readlines(), num2display)
#     for fname in files:
#         fname = os.path.basename(fname.strip())
#         display(df.loc[fname][['target_seq', 'pred_seq']])
#         gold = ipImage(filename=os.path.join(rendered_dir, 'images_gold', fname), unconfined=True)
#         pred = ipImage(filename=os.path.join(rendered_dir, 'images_pred', fname), unconfined=True)
#         display(gold)
#         display(pred)
gold = []; pred = [];
gold_images = []; pred_images = []
with open(os.path.join(rendered_dir, 'unmatched_filenames.txt'), 'r') as f:
    files = np.random.choice(f.readlines(), num2display)
    for fname in files:
        fname = os.path.basename(fname.strip())
        if not os.path.exists(os.path.join(rendered_dir, 'images_pred', fname)):
            continue
        gold_images.append(ipImage(filename=os.path.join(rendered_dir, 'images_gold', fname), format='png', unconfined=True))
        gold.append('<img src=".%s"/>' % os.path.join(rendered_dir, 'images_gold', fname))
        pred_images.append(ipImage(filename=os.path.join(rendered_dir, 'images_pred', fname), format='png', unconfined=True))
        pred.append('<img src=".%s"/>' % os.path.join(rendered_dir, 'images_pred', fname))
df_= pd.DataFrame({'gold': gold, 'pred': pred})
HTML((df_).to_html(escape=False))

In [ ]:
def loadImage(path, mode="RGB"):
    img = Image.open(path)
    img = img.convert(mode)
    return np.asarray(img)

def plotImage(path, axes):
    image_data = loadImage(path)
#     title = os.path.splitext(os.path.basename(path))[0] + ' %s'%(image_data.shape,)
#     axes.set_title( title )
    axes.set_xlim(0,1000)
    axes.imshow(image_data, aspect='equal', extent=None, resample=False, interpolation='bilinear')
    return

def plotImages(image_paths, dpi=None):
    """ image_details should be an array of image path and width - [[path, image_data, image_data.shape], ...] """
    ## Display image data loaded as a numpy array
    try:
        plt.close(fig)
    except:
        pass
    orig_dpi = plt.rcParams['figure.dpi']

    for path in (image_paths):
        gold = os.path.join(rendered_dir, 'images_gold', path)
        pred = os.path.join(rendered_dir, 'images_pred', path)
        image_data.append(plotImage(gold, grid[i]))
        i+=1
        image_data.append(plotImage(pred, grid[i]))
        i+=1
        
    with mpl.rc_context(rc={'figure.dpi': dpi or orig_dpi}):
        fig = plt.figure(figsize=(20.,2.*len(image_paths)))
        grid = ImageGrid(fig, 111, nrows_ncols=(len(image_paths),2), axes_pad=(0.1, 0.5), label_mode="L")
        i = 0
        for path in (image_paths):
            gold = os.path.join(rendered_dir, 'images_gold', path)
            pred = os.path.join(rendered_dir, 'images_pred', path)
            plotImage(gold, grid[i])
            i+=1
            plotImage(pred, grid[i])
            i+=1
    return # image_data

num2display = 5
with open(os.path.join(rendered_dir, 'unmatched_filenames.txt'), 'r') as f:
    files = [os.path.basename(fname.strip()) for fname in np.random.choice(f.readlines(), num2display)]
fnames=[]
plotImages(files)

# fig, axes = plt.subplots(nrows=num2display, ncols=2, sharex=True, sharey=True, squeeze=True)
# for i, fname in enumerate(files):
#     fname = os.path.basename(fname.strip())
#     im_gold = loadImage(os.path.join(rendered_dir, 'images_gold', fname))
#     im_pred = loadImage(os.path.join(rendered_dir, 'images_pred', fname))
#     axes[i][0].imshow(im_gold, aspect='equal', extent=None, resample=False, interpolation='bilinear', cmap='gray')
#     axes[i][1].imshow(im_pred, aspect='equal', extent=None, resample=False, interpolation='bilinear', cmap='gray')

In [ ]:
# for fname in files:
#     print os.path.join(rendered_dir,'images_gold', os.path.basename(fname.strip()))
#     print os.path.join(rendered_dir,'images_pred', os.path.basename(fname.strip()))